1. 문서 내용 읽기
2. 문서 분할(쪼개기)
    - 문서를 분할하지 않으면,
        - 토큰 수 초과로 답변을 생성하지 못할 수 있음
        -문서 길이(input)가 길어서 답변 생성에 시간 소요됨 -> 답변 늦어짐
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있으면, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [4]:
%pip install -Uq langchain langchain-community


Note: you may need to restart the kernel to use updated packages.


In [5]:
## 패키지/라이브러리 설치
# %pip install -qU docx2txt langchain_community langchain-text-splitters langchain-chroma %pip install pinecone %pip install langchain-pinecone

In [12]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone


# 1. 문서 내용 읽고 분할 ########################################
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

# 2. 임베딩 -> 벡터 데이터베이스에 저장 ######################################

## 2-1 환경변수 읽어오기
load_dotenv()

## 2-2. 임베딩 모델 지정 
embedding = OpenAIEmbeddings(model='text-embedding-3-large')


## 3. Pinecone vecter DB
api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

from langchain_pinecone import PineconeVectorStore

database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name='law-index'  
)



## 4. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색 ############################
## 4-1. 사용자 질문
query = '전세사기피해자 금융지원에 대해 알려다오'

## 4-2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query)

## 5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 #############################
## 5-1 프롬프트 작성 
prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
-[context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}

'''
## 5-2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(
    retrieved_docs=retrieved_docs,
    query=query
)


## 5-3. LLM 모델 생성(chatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model='gpt-4o')

## 5-4. LLM 모델에 질문과 검색된 문서를 보냄
ai_message = llm.invoke(formatted_prompt)
ai_message

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************XGEA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}